# Recommender System based on User Ratings

Step 1. Import Libraries

In [2]:
import pandas as pd
import numpy as np
from lenskit.algorithms import Recommender
from lenskit.algorithms.user_knn import UserUser

Step 2: Import Data

In [13]:
def import_data(customers, products, orders, sales):
    #Read in relevant files
    customers_record = pd.read_csv(customers)
    products_record = pd.read_csv(products)
    orders_record = pd.read_csv(orders)
    sales_record = pd.read_csv(sales)

    #Merge all records into one dataframe
    merged_record = pd.merge(customers_record, orders_record, on="customer_id")
    merged_record = pd.merge(merged_record, sales_record, on="order_id")
    merged_record = pd.merge(merged_record, products_record, on="product_id")
    return merged_record

In [14]:
recommender_data = import_data("customers.csv", "products.csv", "orders.csv", "sales.csv")
print(recommender_data.head())

   customer_id  customer_name    gender  age                    home_address  \
0            1  Leanna Busson    Female   30  8606 Victoria TerraceSuite 560   
1            1  Leanna Busson    Female   30  8606 Victoria TerraceSuite 560   
2           81  Lanny Wardale    Female   30     4870 Jackson SummitApt. 538   
3          258  Mira Heinonen    Female   62     2719 Dietrich KnollApt. 175   
4          432  Ettie Gonnely  Bigender   34      6172 Andrew AvenueApt. 194   

   zip_code            city               state    country  order_id  ...  \
0      5464   Johnstonhaven  Northern Territory  Australia       455  ...   
1      5464   Johnstonhaven  Northern Territory  Australia       455  ...   
2      1915  Spencerborough   Western Australia  Australia       863  ...   
3      1252    South Declan          Queensland  Australia       793  ...   
4      7029      North Cody            Victoria  Australia       939  ...   

   price_per_unit quantity_x total_price  product_type  

Step 3. Generate/Process Data (Add User Ratings)

In [5]:
def add_ratings(data):
    #Add random rating between 1 and 5 to each row (sale)
    data['rating'] = np.random.randint(1, 6, size = data.shape[0])
    return data

In [6]:
data_with_ratings = add_ratings(recommender_data)
print(data_with_ratings.head())

   sales_id  order_id  product_id  price_per_unit  quantity_x  total_price  \
0         0         1         218             106           2          212   
1         1         1         481             118           1          118   
2         2         1           2              96           3          288   
3         3         1        1002             106           2          212   
4         4         1         691             113           3          339   

   customer_id  payment order_date delivery_date  ...  \
0           64    30811  2021-8-30    2021-09-24  ...   
1           64    30811  2021-8-30    2021-09-24  ...   
2           64    30811  2021-8-30    2021-09-24  ...   
3           64    30811  2021-8-30    2021-09-24  ...   
4           64    30811  2021-8-30    2021-09-24  ...   

                                   description     customer_name       gender  \
0   A orange coloured, L sized, Chambray Shirt  Annabella Devote  Genderfluid   
1    A indigo coloured, S 

Step 4:  Implement Recommender System

In [10]:
def recommender(data, num_neighbours, min_neighbours):
   
    #Represent ratings
    data = data[['customer_id','product_name','rating']]
    data_copy = data.copy()
    data_copy.rename(columns = {'customer_id' : 'user', 'product_name': 'item'}, inplace= True)

    #Implement reccomender algorithm
    user_user = UserUser(num_neighbours, min_nbrs = min_neighbours) #setting number of neighbours to consider
    algo = Recommender.adapt(user_user)
    algo.fit(data_copy)
    return algo

In [11]:
algo = recommender(data_with_ratings, 15, 3)

Step 5: Print Top 3 Recommendations for a User

In [12]:
#create top 3 recommendations for selected user
user_recs = algo.recommend(1, 3)
print(user_recs)

              item     score
0     High-Waisted  3.479683
1          Pleated  3.307329
2  Casual Slim Fit  3.278656
